In [1]:
import numpy as np
import time

import cv2 as cv
import mediapipe as mp

In [2]:
def countFinger(path = 0, show_fps = False, drawLand = False, min_detection_confidence = 0.75, min_tracking_confidence = 0.5, wCam = 640, hCam = 480):

    video = cv.VideoCapture(0)
    video.set(3, wCam)
    video.set(4, hCam)

    mphands = mp.solutions.hands
    hands = mphands.Hands(max_num_hands = 1, min_detection_confidence = min_detection_confidence, min_tracking_confidence = min_tracking_confidence)
    mpdraw = mp.solutions.drawing_utils

    hand = []

    cTime = 0
    pTime = 0

    while True:
        success, frame = video.read()
        frame = cv.flip(frame, 1)

        if success:

            rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

            result = hands.process(rgb)

            if result.multi_hand_landmarks:
                for handlms in result.multi_hand_landmarks:
                    if drawLand:
                        mpdraw.draw_landmarks(frame, handlms, mphands.HAND_CONNECTIONS)
                    hand = handlms.landmark
            else:
                hand = []

            if(len(hand)>0):

                handfacing = ''
                if(abs(hand[0].y-hand[9].y) > abs(hand[0].x-hand[9].x)):
                    # hand either up or down
                    if(hand[0].y>=hand[9].y):
                        handfacing = 'up'
                    else:
                        handfacing = 'down'
                else:
                    # hand either left or right
                    if(hand[0].x>=hand[9].x):
                        handfacing = 'left'
                    else:
                        handfacing = 'right'

                if(handfacing=='up' or handfacing=='down'):

                    if(handfacing=='down'):
                        for i in range(len(hand)):
                            # making y landmark values negative to work the same as hand facing up
                            hand[i].y = -hand[i].y

                    thumbright = True

                    if(hand[2].x >=hand[0].x):
                        thumbright = True
                    else:
                        thumbright = False

                    if not thumbright:
                        for i in range(len(hand)):
                            # making x landmark values negative to work the same as thumb facing right
                            hand[i].x = -hand[i].x

                    finger = 0
                    for i in range(8,21,4):
                        if(hand[i].y<=hand[i-2].y):
                            # counting fingers up
                            finger+=1
                    if(hand[4].x>=hand[3].x):
                        # counting thumb up
                        finger+=1
                    cv.rectangle(frame, (20,255), (178,425), (0,0,0), -1)
                    cv.putText(frame, str(finger), (50,390), cv.FONT_HERSHEY_COMPLEX, 5, (0,255,255), 5)

                else:

                    if(handfacing=='left'):
                        for i in range(len(hand)):
                            # making x landmark values negative to work the same as hand facing right
                            hand[i].x = -hand[i].x

                    thumbup = True

                    if(hand[2].y <=hand[0].y):
                        thumbup = True
                    else:
                        thumbup = False

                    if not thumbup:
                        for i in range(len(hand)):
                            # making y landmark values negative to work the same as thumb facing up
                            hand[i].y = -hand[i].y

                    finger = 0
                    for i in range(8,21,4):
                        if(hand[i].x>=hand[i-2].x):
                            # counting fingers up
                            finger+=1
                    if(hand[4].y<=hand[3].y):
                        # counting thumb up
                        finger+=1
                    cv.rectangle(frame, (20,255), (178,425), (0,0,0), -1)
                    cv.putText(frame, str(finger), (50,390), cv.FONT_HERSHEY_COMPLEX, 5, (0,255,255), 5)

            else:
                cv.putText(frame, 'Hand not found', (15,80), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)

            if show_fps:
                cTime = time.time()
                fps = int(1/(cTime-pTime))
                pTime = cTime
                cv.putText(frame, 'FPS: '+str(fps), (10,40), cv.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)

            cv.imshow('Video', frame)

            if(cv.waitKey(1) & 0xFF==ord('d')):
                break

        else:
            break

    video.release()
    cv.destroyAllWindows()

## countFinger Method

* **path -** path of the video source. 0: webcam / 1,2,..: connected media number / '../path': video file path. (default: 0)
* **show_fps -** show fps of running video. (default: False)
* **drawLand -** draw hand landmarks. (default: False)
* **min_detection_confidence -** minimum detection confidence. (default: 0.75)
* **min_tracking_confidence -** minimum tracking confidence. (default: 0.5)
* **wCam -** set width of captured video source. (default: 640)
* **hCam -** set height of captured video source. (default: 480)

In [3]:
countFinger(show_fps = True)